In [1]:
from google.cloud import aiplatform

project_id = "dataanalytics-347914"
region = "us-central1"
staging_bucket="gs://udemy-gcp-mlops"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

<h4> Upload the stable version to model registry

In [2]:
display_name = "coupon-recommendation-model"
artifact_uri = "gs://udemy-gcp-mlops/coupon-recommendation/artifacts/"
serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest"

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        sync=False,
        version_aliases=["stable-version"],
        is_default_version=True
    )

model.wait()

Creating Model
Create Model backing LRO: projects/936546808722/locations/us-central1/models/8548879927330471936/operations/4079877704638267392
Model created. Resource name: projects/936546808722/locations/us-central1/models/8548879927330471936@1
To use this Model in another session:
model = aiplatform.Model('projects/936546808722/locations/us-central1/models/8548879927330471936@1')


In [3]:
# DISPLAY_NAME = "coupon-recommendation-model"
# models = aiplatform.Model.list(filter=("display_name={}").format(DISPLAY_NAME))
# parent_model = models[0].resource_name

<h4> Upload the tuned V1 version to the model registry with <b>parent_model</b> parameter

In [5]:
project_id = 936546808722
model_id = 8548879927330471936
parent_model = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)

new_model = aiplatform.Model.upload(
        parent_model = parent_model,
        artifact_uri="gs://udemy-gcp-mlops/coupon-recommendation/hpt-tuned-artifacts/",
        serving_container_image_uri=serving_container_image_uri,
        sync=False,
        version_aliases=["v2"],
        labels={"release":"dev"},
        is_default_version=False
    )
new_model.wait()

<h4> Deploy the stable model to an endpoint

In [6]:
deployed_model_display_name = "coupon-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

Creating Endpoint
Create Endpoint backing LRO: projects/936546808722/locations/us-central1/endpoints/1186910707556286464/operations/4613554260481671168
Endpoint created. Resource name: projects/936546808722/locations/us-central1/endpoints/1186910707556286464
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/936546808722/locations/us-central1/endpoints/1186910707556286464')
Deploying model to Endpoint : projects/936546808722/locations/us-central1/endpoints/1186910707556286464
Deploy Endpoint model backing LRO: projects/936546808722/locations/us-central1/endpoints/1186910707556286464/operations/8907736505179439104
Endpoint model deployed. Resource name: projects/936546808722/locations/us-central1/endpoints/1186910707556286464


<h4>Run Batch Predictions against stable model version

In [ ]:
model = aiplatform.Model('projects/936546808722/locations/us-central1/models/8548879927330471936@1')

gcs_input_uri = 'gs://udemy-gcp-mlops/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://udemy-gcp-mlops/coupon-recommendation/batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False
)

<h4>Run Batch Predictions against V2 model version

In [ ]:
new_model = aiplatform.Model('projects/936546808722/locations/us-central1/models/8548879927330471936@3')

gcs_input_uri = 'gs://udemy-gcp-mlops/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://udemy-gcp-mlops/coupon-recommendation/batch-prediction-output"

batch_predict_job = new_model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False
)